In [1]:
system"c 40 200"

In [2]:
complete:("PJFFFFFFFFFJFJJF";enlist ",")0:`$":complete.csv";

In [3]:
final:select from complete where WEEKDAY<5,9<HOUR,HOUR<20

In [4]:
final:select from final where 40 <= (avg;traffic_load) fby traffic_station

In [5]:
final:update traffic_load:traffic_load%100 from final

In [6]:
minMaxScale:{[l]
    minL:min l;
    maxL:max l;
    ({(x-y)%(z-y)}[;minL;maxL]')l}

In [7]:
scaledRainfall:minMaxScale final[`rainfall]
scaledTemperature:minMaxScale final[`temperature]

final[`rainfall]:scaledRainfall
final[`temperature]:scaledTemperature

In [8]:
show scaledRainfall
show scaledTemperature

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0..
0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.26666..


In [9]:
time_window:{[tt;data]
    op:$[tt=`train;#;_];                                      / `train or `test decide the operator
    m:`rainfall`temperature`traffic_load`HOUR`WEEKDAY;        / the 5 columns we need
    data:?[data;();`traffic_station;m!({(y;(-;(count;x);80);x)}[;op]')m]; / first 80 or until the last 80 depending on operator 
    sw:{({y#z _x}[x;5;]')til count b:6_x};                    / sliding window function. takes turbomatrix and divides into chunks of 5x5
    gl:{6_(flip x)[2]};                                       / gets the load (y data)
    toMatrix:{({[t;i]value t[i]}[x;]')til count x:flip x};    / table to matrix
    data:(toMatrix')data;                                     / convert each subtable (data is a keyed table) to a matrix
    X:(sw')data;                                              / apply sliding window to get X
    y:(gl')data;                                              / and gl to get y
    (X;y)                                                     / return both
    }

In [10]:
train:time_window[`train;final]

In [11]:
train[0][3403][0]

0f 0.2242424 0.4975 10 3
0f 0.3333333 0.525  11 3
0f 0.4424242 0.4725 12 3
0f 0.5151515 0.4975 13 3
0f 0.5818182 0.58   14 3


In [12]:
last train[0][3403]

0f 0.4       0.3875 17 0
0f 0.3212121 0.515  18 0
0f 0.3030303 0.34   19 0
0f 0.3212121 0.465  10 1
0f 0.2666667 0.5125 11 1


In [13]:
train[1][3403]

0.4875 0.4725 0.425 0.475 0.5425 0.4725 0.4525 0.4675 0.58 0.4225 0.3925 0.4925 0.5625 0.5775 0.3925 0.485 0.4325 0.57 0.49 0.3325 0.37 0.4325 0.4425 0.4475 0.26 0.445 0.355 0.44 0.4775 0.33 0.1975..


In [13]:
test:time_window[`test;final]

[0;31mprepareData[0m: [0;31mprepareData[0m

In [13]:
test[0][3403][0]

[0;31mtest[0m: [0;31mtest[0m

In [13]:
last test[0][3403]

[0;31mtest[0m: [0;31mtest[0m

In [13]:
test[1][3403]

[0;31mtest[0m: [0;31mtest[0m